In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AI Platform (Unified) SDK: AutoML natural language text classification model




## Installation

Install the latest (preview) version of AI Platform (Unified) SDK.

In [ ]:
! pip3 install -U google-cloud-aiplatform --user


Install the Google *cloud-storage* library as well.

In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AI Platform (Unified) SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AI Platform Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AI Platform (Unified). We recommend when possible, to choose the region closest to you. 

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AI Platform. Not all regions provide support for all AI Platform services. For the latest support per region, see [Region support for AI Platform (Unified) services](https://cloud.google.com/ai-platform-unified/docs/general/locations)

In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AI Platform notebook and run the cell, the cell knows to skip executing the authentication steps.*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets. 

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME
    

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME
    

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

#### Import AI Platform (Unified) SDK

Import the AI Platform (Unified) SDK into our Python environment.

In [ ]:
import base64
import json
import os
import sys
import time


from google.cloud.aiplatform import gapic as aip


from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from google.protobuf.struct_pb2 import Struct
from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import ParseDict


#### AI Platform (Unified) constants

Setup up the following constants for AI Platform (Unified):

- `API_ENDPOINT`: The AI Platform (Unified) API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The AI Platform (Unified) location root path for dataset, model and endpoint resources.

In [ ]:
# API Endpoint
API_ENDPOINT = "{0}-aiplatform.googleapis.com".format(REGION)

# AI Platform (Unified) location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


#### AutoML constants

Next, setup constants unique to AutoML Text Classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the managed pipelines service the task (e.g., classification) to train the model for.

In [ ]:
# Text Dataset type
TEXT_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
# Text Labeling type
IMPORT_SCHEMA_TEXT_CLASSIFICATION = "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_classification_single_label_io_format_1.0.0.yaml"
# Text Training task
TRAINING_TEXT_CLASSIFICATION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_classification_1.0.0.yaml"


## Clients AI Platform (Unified)

The AI Platform (Unified) SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AI Platform).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(
        client_options=client_options
    )
    return client


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(
        client_options=client_options
    )
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(
        client_options=client_options
    )
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(
        client_options=client_options
    )
    return client

def create_job_client():
    client = aip.JobServiceClient(
        client_options=client_options
    )
    return client


clients = {}
clients['dataset'] = create_dataset_client()
clients['model'] = create_model_client()
clients['pipeline'] = create_pipeline_client()
clients['endpoint'] = create_endpoint_client()
clients['prediction'] = create_prediction_client()
clients['job'] = create_job_client()

for client in clients.items():
    print(client)


In [ ]:
IMPORT_FILE = 'gs://cloud-ml-data/NL-classification/happiness.csv'


In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10 


*Example output*:
```
I went on a successful date with someone I felt sympathy and connection with.,affection
I was happy when my son got 90% marks in his examination,affection
I went to the gym this morning and did yoga.,exercise
We had a serious talk with some friends of ours who have been flaky lately. They understood and we had a good evening hanging out.,bonding
I went with grandchildren to butterfly display at Crohn Conservatory,affection
I meditated last night.,leisure
"I made a new recipe for peasant bread, and it came out spectacular!",achievement
I got gift from my elder brother which was really surprising me,affection
YESTERDAY MY MOMS BIRTHDAY SO I ENJOYED,enjoy_the_moment
Watching cupcake wars with my three teen children,affection
```



## Create a dataset

### [projects.locations.datasets.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.datasets/create)

#### Request

In [ ]:
DATA_SCHEMA = TEXT_SCHEMA

dataset = {
    "display_name": "happiness_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(
            parent=PARENT,
            dataset=dataset
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "happiness_20210226015238",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
  }
}
```



#### Call

In [ ]:
request = clients['dataset'].create_dataset(
    parent=PARENT, 
    dataset=dataset
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/574578388396670976",
  "displayName": "happiness_20210226015238",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "TEXT"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/text_1.0.0.yaml"
  }
}
```



In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)


### [projects.locations.datasets.import](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.datasets/import)

#### Request

In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_TEXT_CLASSIFICATION

import_config = {
    "gcs_source": {
        "uris": [IMPORT_FILE]
    },
    "import_schema_uri": LABEL_SCHEMA
}

print(MessageToJson(
    aip.ImportDataRequest(
        name=dataset_short_id,
        import_configs=[import_config]
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "name": "574578388396670976",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://cloud-ml-data/NL-classification/happiness.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_classification_single_label_io_format_1.0.0.yaml"
    }
  ]
}
```



#### Call

In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id,
    import_configs=[import_config]
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```



## Train a model

### [projects.locations.trainingPipelines.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

#### Request

In [ ]:
TRAINING_SCHEMA = TRAINING_TEXT_CLASSIFICATION_SCHEMA

task = json_format.ParseDict({'multi_label': False,},Value())

training_pipeline = {
    "display_name": "happiness_" + TIMESTAMP,
    "input_data_config": {
        "dataset_id": dataset_short_id
    },
    "model_to_upload": {
        "display_name": "happiness_" + TIMESTAMP
    },
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT,
            training_pipeline=training_pipeline
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "happiness_20210226015238",
    "inputDataConfig": {
      "datasetId": "574578388396670976"
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_classification_1.0.0.yaml",
    "trainingTaskInputs": {
      "multi_label": false
    },
    "modelToUpload": {
      "displayName": "happiness_20210226015238"
    }
  }
}
```



#### Call

In [ ]:
request = clients['pipeline'].create_training_pipeline(
    parent=PARENT, 
    training_pipeline=training_pipeline
)


#### Response

In [ ]:
print(
    MessageToJson(
        request.__dict__["_pb"]
    )
)


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/2903115317607661568",
  "displayName": "happiness_20210226015238",
  "inputDataConfig": {
    "datasetId": "574578388396670976"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_classification_1.0.0.yaml",
  "trainingTaskInputs": {},
  "modelToUpload": {
    "displayName": "happiness_20210226015238"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T02:23:54.166560Z",
  "updateTime": "2021-02-26T02:23:54.166560Z"
}
```



In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split('/')[-1]

print(training_pipeline_id)


### [projects.locations.trainingPipelines.get](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

#### Call

In [ ]:
request = clients["pipeline"].get_training_pipeline(
    name=training_pipeline_id
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/2903115317607661568",
  "displayName": "happiness_20210226015238",
  "inputDataConfig": {
    "datasetId": "574578388396670976"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_classification_1.0.0.yaml",
  "trainingTaskInputs": {},
  "modelToUpload": {
    "name": "projects/116273516712/locations/us-central1/models/2369051733671280640",
    "displayName": "happiness_20210226015238"
  },
  "state": "PIPELINE_STATE_SUCCEEDED",
  "createTime": "2021-02-26T02:23:54.166560Z",
  "startTime": "2021-02-26T02:23:54.396088Z",
  "endTime": "2021-02-26T06:08:06.548524Z",
  "updateTime": "2021-02-26T06:08:06.548524Z"
}
```



In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_name = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(20)

print(model_id)


## Evaluate the model

### [projects.locations.models.evaluations.list](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

#### Call

In [ ]:
request = clients["model"].list_model_evaluations(
    parent=model_id
)


#### Response

In [ ]:
model_evaluations = [
    json.loads(MessageToJson(mel.__dict__["_pb"])) for mel in request 
]

print(json.dumps(model_evaluations, indent=2))

# The evaluation slice
evaluation_slice = request.model_evaluations[0].name


*Example output*:
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/2369051733671280640/evaluations/1541152463304785920",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    "metrics": {
      "confusionMatrix": {
        "annotationSpecs": [
          {
            "displayName": "exercise",
            "id": "952213353537732608"
          },
          {
            "id": "1528674105841156096",
            "displayName": "achievement"
          },
          {
            "id": "3258056362751426560",
            "displayName": "leisure"
          },
          {
            "id": "3834517115054850048",
            "displayName": "bonding"
          },
          {
            "id": "5563899371965120512",
            "displayName": "enjoy_the_moment"
          },
          {
            "id": "6140360124268544000",
            "displayName": "nature"
          },
          {
            "id": "8446203133482237952",
            "displayName": "affection"
          }
        ],
        "rows": [
          [
            19.0,
            1.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0
          ],
          [
            0.0,
            342.0,
            5.0,
            2.0,
            13.0,
            2.0,
            13.0
          ],
          [
            2.0,
            10.0,
            42.0,
            1.0,
            12.0,
            0.0,
            2.0
          ],
          [
            0.0,
            4.0,
            0.0,
            121.0,
            1.0,
            0.0,
            4.0
          ],
          [
            2.0,
            29.0,
            3.0,
            2.0,
            98.0,
            0.0,
            6.0
          ],
          [
            0.0,
            3.0,
            0.0,
            1.0,
            0.0,
            21.0,
            1.0
          ],
          [
            0.0,
            7.0,
            0.0,
            1.0,
            6.0,
            0.0,
            409.0
          ]
        ]
      },
      "confidenceMetrics": [
        {
          "f1Score": 0.25,
          "recall": 1.0,
          "f1ScoreAt1": 0.88776374,
          "precisionAt1": 0.88776374,
          "precision": 0.14285715,
          "recallAt1": 0.88776374
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 0.9721519,
          "f1Score": 0.8101266,
          "recallAt1": 0.88776374,
          "f1ScoreAt1": 0.88776374,
          "precisionAt1": 0.88776374,
          "precision": 0.69439423
        },
        
        # REMOVED FOR BREVITY
        
        {
          "f1Score": 0.0033698399,
          "recall": 0.0016877637,
          "confidenceThreshold": 1.0,
          "recallAt1": 0.0016877637,
          "f1ScoreAt1": 0.0033698399,
          "precisionAt1": 1.0,
          "precision": 1.0
        }
      ],
      "auPrc": 0.95903283,
      "logLoss": 0.08260541
    },
    "createTime": "2021-02-26T06:07:48.967028Z",
    "sliceDimensions": [
      "annotationSpec"
    ]
  }
]
```



### [projects.locations.models.evaluations.get](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)

#### Call

In [ ]:
request = clients["model"].get_model_evaluation(
    name=evaluation_slice
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/2369051733671280640/evaluations/1541152463304785920",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
  "metrics": {
    "confusionMatrix": {
      "annotationSpecs": [
        {
          "displayName": "exercise",
          "id": "952213353537732608"
        },
        {
          "displayName": "achievement",
          "id": "1528674105841156096"
        },
        {
          "id": "3258056362751426560",
          "displayName": "leisure"
        },
        {
          "id": "3834517115054850048",
          "displayName": "bonding"
        },
        {
          "displayName": "enjoy_the_moment",
          "id": "5563899371965120512"
        },
        {
          "displayName": "nature",
          "id": "6140360124268544000"
        },
        {
          "id": "8446203133482237952",
          "displayName": "affection"
        }
      ],
      "rows": [
        [
          19.0,
          1.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0
        ],
        [
          0.0,
          342.0,
          5.0,
          2.0,
          13.0,
          2.0,
          13.0
        ],
        [
          2.0,
          10.0,
          42.0,
          1.0,
          12.0,
          0.0,
          2.0
        ],
        [
          0.0,
          4.0,
          0.0,
          121.0,
          1.0,
          0.0,
          4.0
        ],
        [
          2.0,
          29.0,
          3.0,
          2.0,
          98.0,
          0.0,
          6.0
        ],
        [
          0.0,
          3.0,
          0.0,
          1.0,
          0.0,
          21.0,
          1.0
        ],
        [
          0.0,
          7.0,
          0.0,
          1.0,
          6.0,
          0.0,
          409.0
        ]
      ]
    },
    "logLoss": 0.08260541,
    "confidenceMetrics": [
      {
        "precision": 0.14285715,
        "precisionAt1": 0.88776374,
        "recall": 1.0,
        "f1ScoreAt1": 0.88776374,
        "recallAt1": 0.88776374,
        "f1Score": 0.25
      },
      {
        "f1Score": 0.8101266,
        "recall": 0.9721519,
        "precision": 0.69439423,
        "confidenceThreshold": 0.05,
        "recallAt1": 0.88776374,
        "precisionAt1": 0.88776374,
        "f1ScoreAt1": 0.88776374
      },
      
      # REMOVED FOR BREVITY
      
      {
        "confidenceThreshold": 1.0,
        "f1Score": 0.0033698399,
        "f1ScoreAt1": 0.0033698399,
        "precisionAt1": 1.0,
        "precision": 1.0,
        "recall": 0.0016877637,
        "recallAt1": 0.0016877637
      }
    ],
    "auPrc": 0.95903283
  },
  "createTime": "2021-02-26T06:07:48.967028Z",
  "sliceDimensions": [
    "annotationSpec"
  ]
}
```



## Make batch predictions

### Prepare files for batch prediction

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(',')

print(test_item, test_label)


*Example output*:
```
I went on a successful date with someone I felt sympathy and connection with. affection
```



### Make the batch input file

Let's now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each text file. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the text file.
- `mimeType`: The content type. In our example, it is an `text/plain` file.


In [ ]:
import tensorflow as tf
import json


test_item_uri = "gs://" + BUCKET_NAME + "/test.txt"
with tf.io.gfile.GFile(test_item_uri, 'w') as f:
    f.write(test_item + '\n')

gcs_input_uri = "gs://" + BUCKET_NAME + '/test.jsonl'
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    data = {"content": test_item_uri, "mime_type": "text/plain"}
    f.write(json.dumps(data) + '\n')


In [ ]:
! gsutil cat $gcs_input_uri
! gsutil cat $test_item_uri


*Example output*:
```
{"content": "gs://f33-ucaip-trainingaip-20210226015238/test.txt", "mime_type": "text/plain"}
I went on a successful date with someone I felt sympathy and connection with.
```



### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)

#### Request

In [ ]:
batch_prediction_job = {
    "display_name": "happiness_" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {
            "uris": [gcs_input_uri]
        }
    },
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        }
    },
    "dedicated_resources": {
        "machine_spec": {
            "machine_type": "n1-standard-2",
            "accelerator_count": 0,
        },
        "starting_replica_count": 1,
        "max_replica_count": 1
    }
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT,
            batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "happiness_20210226015238",
    "model": "projects/116273516712/locations/us-central1/models/2369051733671280640",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://f33-ucaip-trainingaip-20210226015238/test.jsonl"
        ]
      }
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210226015238/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```



#### Call

In [ ]:
request = clients['job'].create_batch_prediction_job(
    parent=PARENT, 
    batch_prediction_job=batch_prediction_job
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/4770983263059574784",
  "displayName": "happiness_20210226015238",
  "model": "projects/116273516712/locations/us-central1/models/2369051733671280640",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://f33-ucaip-trainingaip-20210226015238/test.jsonl"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210226015238/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T09:37:44.471843Z",
  "updateTime": "2021-02-26T09:37:44.471843Z"
}
```



In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split('/')[-1]

print(batch_job_id)


### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)

#### Call

In [ ]:
request = clients['job'].get_batch_prediction_job(
    name=batch_job_id
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/4770983263059574784",
  "displayName": "happiness_20210226015238",
  "model": "projects/116273516712/locations/us-central1/models/2369051733671280640",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://f33-ucaip-trainingaip-20210226015238/test.jsonl"
      ]
    }
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210226015238/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T09:37:44.471843Z",
  "updateTime": "2021-02-26T09:37:44.471843Z"
}
```



In [ ]:
def get_latest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest

while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(response.output_config.gcs_destination.output_uri_prefix)
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)


*Example output*:
```
gs://f33-ucaip-trainingaip-20210226015238/batch_output/prediction-happiness_20210226015238-2021-02-26T09:37:44.261133Z/predictions_00001.jsonl
{"instance":{"content":"gs://f33-ucaip-trainingaip-20210226015238/test.txt","mimeType":"text/plain"},"prediction":{"ids":["8446203133482237952","3834517115054850048","1528674105841156096","5563899371965120512","952213353537732608","3258056362751426560","6140360124268544000"],"displayNames":["affection","bonding","achievement","enjoy_the_moment","exercise","leisure","nature"],"confidences":[0.9183423,0.045685068,0.024327256,0.0057157497,0.0040851077,0.0012627868,5.8173126E-4]}}
```



## Make online predictions

### [projects.locations.endpoints.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/create)

#### Request

In [ ]:
endpoint = {
    "display_name": "happiness_" + TIMESTAMP
}

print(
    MessageToJson(
        aip.CreateEndpointRequest(
            parent=PARENT, 
            endpoint=endpoint
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "happiness_20210226015238"
  }
}
```



#### Call

In [ ]:
request = clients['endpoint'].create_endpoint(
    parent=PARENT, 
    endpoint=endpoint
)


#### Response

In [ ]:
result =request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/7367713068517687296"
}
```



In [ ]:
# The fully qualified ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split('/')[-1]

print(endpoint_id)


### [projects.locations.endpoints.deployModel](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)

#### Request

In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "happiness_" + TIMESTAMP,
    "automatic_resources": {
        "min_replica_count": 1, 
        "max_replica_count": 1
    }
}

traffic_split={"0": 100}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id, 
            deployed_model=deployed_model, 
            traffic_split=traffic_split            
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/7367713068517687296",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/2369051733671280640",
    "displayName": "happiness_20210226015238",
    "automaticResources": {
      "minReplicaCount": 1,
      "maxReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```



#### Call

In [ ]:
request = clients['endpoint'].deploy_model(
    endpoint=endpoint_id, 
    deployed_model=deployed_model, 
    traffic_split=traffic_split
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "deployedModel": {
    "id": "418518105996656640"
  }
}
```



In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)


### [projects.locations.endpoints.predict](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)

#### Request

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(',')

instances_list = [{"content": test_item}]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]

request = aip.PredictRequest(
    endpoint=endpoint_id, 
)
request.instances.append(instances)

print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/7367713068517687296",
  "instances": [
    [
      {
        "content": "I went on a successful date with someone I felt sympathy and connection with."
      }
    ]
  ]
}
```



#### Call

In [ ]:
request = clients['prediction'].predict(
    endpoint=endpoint_id, 
    instances=instances
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "predictions": [
    {
      "confidences": [
        0.8867673277854919,
        0.024743923917412758,
        0.0034913308918476105,
        0.07936617732048035,
        0.0013463868526741862,
        0.0002393187169218436,
        0.0040455833077430725
      ],
      "displayNames": [
        "affection",
        "achievement",
        "enjoy_the_moment",
        "bonding",
        "leisure",
        "nature",
        "exercise"
      ],
      "ids": [
        "8446203133482237952",
        "1528674105841156096",
        "5563899371965120512",
        "3834517115054850048",
        "3258056362751426560",
        "6140360124268544000",
        "952213353537732608"
      ]
    }
  ],
  "deployedModelId": "418518105996656640"
}
```



### [projects.locations.endpoints.undeployModel](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)



#### Call



In [ ]:
request = clients['endpoint'].undeploy_model(
    endpoint=endpoint_id, 
    deployed_model_id=deployed_model_id, 
    traffic_split={}
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the dataset using the AI Platform (Unified) fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients['dataset'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the AI Platform (Unified) fully qualified identifier for the model
try:
    if delete_model:
        clients['model'].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the AI Platform (Unified) fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients['endpoint'].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the AI Platform (Unified) fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients['pipeline'].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the AI Platform (Unified) fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients['job'].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
